# Postprocessing for mizuRoute global Hanasaki simulations
## Evaluation with GSIM indices

postprocessing from Cheyenne - global simulations with HDMA topology

Inne Vanderkelen - March 2021

**TO DO:**
* Manually check offset location GSIM stations
* double check joined gdf is correct? 
* fix KGE metric (problem with correlation calculation of pandas dataframe)

In [1]:
# import modules
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import os
import hydroeval as he
import pickle
import utils
import warnings
import cartopy as cart
warnings.filterwarnings('ignore')

# plot settings
utils.set_plot_param()

In [5]:
### Settings
intersect_stations = False
classify_controlled = False # for all station, classify controlled stations based on downstream distance from reservoir. If flase, load values
plot = False
calc_dict=True
# only use stations defined as controlled
flag_only_controlled = True
calc_stations = False
sort_indices = True

In [3]:
### Initialisation

# model directory
outdir = '/glade/work/ivanderk/mizuRoute_global/route/'

# current working directory
scriptsdir = os.getcwd() + '/'

# observations dir
datadir = '/glade/work/ivanderk/data/'

# give simulation name
casename= 'natlake'
case_H06 =  'H06'
case_nolake =  'nolake'
case_natlake =  'natlake'

cases = [case_nolake,case_natlake,case_H06]
cases_longname = [  'no lakes','natural lakes','Hanasaki']
colors = [ 'skyblue','yellowgreen', 'coral']

# simulation years
# simulation years
nspinup = 2 # number of spin up years 
start_year = 1979 + nspinup
end_year = 2000

timestep = 'monthly' # monthly # yearly

units = {'KGE': '-', 'NSE': '-', 'RMSE': 'm³/s', 'MARE': '-','PBIAS': '%'}


In [4]:
### Load data

# open river topology as dataset (immediately correct spaces before PFAF code)
ntopo = xr.open_dataset(outdir+'ancillary_data/ntopo_hdma_old_mod.reorder_lake_H06.nc')
#ntopo = xr.open_dataset(outdir+'ancillary_data/ntopo_hdma_mod.reorder_lake_H06.nc')
df_metadata = pd.read_csv(datadir+'gsim/gsim_metadata/GSIM_catalog/GSIM_metadata.csv')


## 1. Prepare GSIM obs for model evaluation


### 1.1 Get GSIM stations that are on river network
if flag intersect_stations is true: do intersection and save files 
if not, just load csv with list of intersecting stations

In [6]:
# settings
# relate GSIM stations to river segments based on offset 
offset = 0.002 # degrees (~200 m on equator)

# threshold reservoir influence
threshold_res_influence = 200e3 # m 

In [6]:
%%time
if intersect_stations: # calculate intersection, otherwise load

    # open gsim meta data, correct for suspect coordinates, and convert into geodataframe
    df_suspect_coordinates =  pd.read_csv(datadir+'gsim/gsim_metadata/GSIM_catalog/GSIM_suspect_coordinates_stations.csv')
    df_metadata = df_metadata[~df_metadata['gsim.no'].isin(df_suspect_coordinates['gsim.no'])]
    gdf_stations_raw = gpd.GeoDataFrame(df_metadata, geometry=gpd.points_from_xy(df_metadata.longitude, df_metadata.latitude))

    # open river topology as geodataframe (necessary for intersection)
    gdf_river = gpd.read_file(datadir+'topology/river_with_lake_flag4_reorder/river_with_lake_flag4_reorder.shp')
    #gdf_river = gpd.read_file(datadir+'topology/HDMA_hydrolakes10km_reorder/river_with_lake_flag4_10km_reorder.shp')
    
    stations_buffer = gdf_stations_raw
    stations_buffer['geometry'] = stations_buffer.buffer(offset)
    joined = gpd.sjoin(stations_buffer,gdf_river, how='inner', op='intersects')
    gdf_stations_intersect = gdf_stations_raw.loc[gdf_stations_raw['gsim.no'].isin(joined['gsim.no'])]
    print(str(gdf_stations_intersect.shape[0])+' of '+str(gdf_stations_raw.shape[0])+' stations on river network ('+str(round(gdf_stations_intersect.shape[0]/gdf_stations_raw.shape[0]*100,1))+'%)')

    # save merged info in CSV
    joined.drop(['geometry'], axis='columns', inplace=True)
    joined.to_csv(datadir+'gsim/stations/stations_HDMA_intersect.csv')
    
    
    # save intersecting stations in shapefile
    gdf_stations_intersect.to_file(datadir+'gsim/stations/stations_HDMA_intersect.shp')
    

    stations = joined
    
else: 
    
    stations = pd.read_csv(datadir+'gsim/stations/stations_HDMA_intersect.csv')

CPU times: user 77.3 ms, sys: 16.4 ms, total: 93.7 ms
Wall time: 119 ms


### 1.2 Classify GSIM stations as reservoir controlled and natural flow
by adding extra column to stations metadata 'controlled' with 1 if yes, and 0 if no

In [10]:
# classify GSIM stations as controlled river stream and naturalised
if classify_controlled: 

    # get pfafs corresponding to segments to hrus trough hru_seg_id
    seg_pfafs = ntopo[['PFAF','seg_id','lake_model']].to_dataframe()
    # correct for spaces in front of the PFAF code
    seg_pfafs['PFAF'] = np.char.strip( seg_pfafs['PFAF'].values.astype(str))
    res_pfaf = seg_pfafs.loc[seg_pfafs['lake_model']==2,'PFAF']

    res_is_station_pfaf = res_pfaf[res_pfaf.isin(stations['PFAF'])]

    # for every reservoir pfaf get downstream pfafs along threshold
    resstream_pfaf = utils.get_rescontrolled_pfafs(ntopo,res_pfaf.values,threshold_res_influence)

    # assign controlled stations
    stations.loc[stations['PFAF'].isin(resstream_pfaf), 'controlled'] = 1

    # save as csv
    stations.to_csv(datadir+'gsim/stations/stations_HDMA_controlled.csv')

    # save as shpfile 
    gdf_stations_intersect = gpd.read_file(datadir+'gsim/stations/stations_HDMA_intersect.shp')
    gdf_stations_intersect.merge(stations,on='gsim.no').to_file(datadir+'gsim/stations/stations_HDMA_intersect_controlled.shp')
    stations_controlled = stations.loc[stations.controlled==1]

    
else: 
    stations = pd.read_csv(datadir+'gsim/stations/stations_HDMA_controlled.csv')
    stations_controlled = stations.loc[stations.controlled==1]


## 2. Compare GSIM with simulation

### 2.1 Get GSIM stations that include modeled time period

In [9]:
start_year_obs = pd.to_datetime(stations['year.start'], format='%Y')
end_year_obs = pd.to_datetime(stations['year.end'], format='%Y')
latest_start = stations['year.start'].apply(lambda x: max(x,start_year))
earliest_end = stations['year.end'].apply(lambda x: min(x,end_year))

stations = stations[(earliest_end-latest_start) >0]
print('Number of stations in modeled period: '+str(stations.shape[0]))

Number of stations in modeled period: 10233


### 2.2 Load mizuRoute simulation masked for GSIM stations

In [23]:
%%time

if calc_dict:
    
    # for different cases 
    # create mask with segids for grdc stations that are classified as controlled
    mask_stations = ntopo['seg_id'].astype('int64').isin(list(stations_controlled['seg_id'].values)) 

    #initialise empty dictionary for storing mizuroute simulations
    ds_sim_dict = {}

    for case in cases: 
        print(case)
        # laod all years and append into one dataset with outflow
        ds_res_year_all =  []
        for year in range(start_year,end_year+1):
            print('Loading year '+str(year)+'\r')
            # Open simulation for one year
            ds = xr.open_dataset(outdir+"output/"+case+".mizuRoute.h."+str(year)+"-01-01-00000.nc")

            # extract reservoir outflow 
            ds_res_year = ds['IRFroutedRunoff'].where(mask_stations).to_dataset(name='discharge')

            ds_res_year_all.append(ds_res_year)

        ds_mod = xr.concat(ds_res_year_all, dim='time')
        ds_sim_dict[case] = ds_mod

nolake
Loading year 1981
Loading year 1982
Loading year 1983
Loading year 1984
Loading year 1985
Loading year 1986
Loading year 1987
Loading year 1988
Loading year 1989
Loading year 1990
Loading year 1991
Loading year 1992
Loading year 1993
Loading year 1994
Loading year 1995
Loading year 1996
Loading year 1997
Loading year 1998
Loading year 1999
Loading year 2000
natlake
Loading year 1981
Loading year 1982
Loading year 1983
Loading year 1984
Loading year 1985
Loading year 1986
Loading year 1987
Loading year 1988
Loading year 1989
Loading year 1990
Loading year 1991
Loading year 1992
Loading year 1993
Loading year 1994
Loading year 1995
Loading year 1996
Loading year 1997
Loading year 1998
Loading year 1999
Loading year 2000
H06
Loading year 1981
Loading year 1982
Loading year 1983
Loading year 1984
Loading year 1985
Loading year 1986
Loading year 1987
Loading year 1988
Loading year 1989
Loading year 1990
Loading year 1991
Loading year 1992
Loading year 1993
Loading year 1994
Loading y

### 2.3 Load GSIM observations per station and compare with mizuRoute simulation

In [24]:
calc_dict=True

In [25]:
%%time

if calc_dict: 
    obs_dict = {}
    mod_dict = {}
    metric_dict = {}

    for i, gsim_no in enumerate(stations_controlled['gsim.no']): 
        #i = 1

        print('Processing station '+str(i)+ ' of '+str(len(stations_controlled['gsim.no'])),end='\r' )


        if timestep=='monthly': 
            fn = datadir+'gsim/gsim_indices/TIMESERIES/monthly/'+gsim_no+'.mon'
            freq='M'
        elif timestep == 'yearly': 
            fn = datadir+'gsim/gsim_indices/TIMESERIES/yearly/'+gsim_no+'.year'
            freq='Y'
        elif timestep == 'seasonal': 
            fn = datadir+'gsim/gsim_indices/TIMESERIES/seasonal/'+gsim_no+'.seas'

        if not os.path.isfile(fn):
            print('No '+timestep+' observations for station '+str(gsim_no))
        else:
            try:
                df_obs = pd.read_csv(fn,header=21, delimiter=',\t')
                df_obs.rename(columns={'"date"':'date','"MEAN"':'MEAN','"SD"':'SD','"CV"':'CV','"IQR"':'IQR','"MIN"':'MIN','"MAX"':'MAX','"MIN7"':'MIN7','"MAX7"':'MAX7','"n.missing"':'n.missing','"n.available"':'n.available'}, inplace=True)
                df_obs['date'] = pd.to_datetime(df_obs['date']).apply(lambda x: x if x.month != 2 and x.date != 29 else pd.datetime(x.year, x.month, 28)) # convert to datetime and replace all leap days (mizuroute does not have leap days)
                df_obs['date'] = df_obs['date']
                df_obs = df_obs.set_index('date')
            except: 
                print('Error loading '+timestep+' obs for station '+str(gsim_no))

            # get obs and mod overlapping timeseries
            sim_period = ds_sim_dict[cases[0]].indexes['time'].to_datetimeindex()
            period_overlap = sim_period.intersection(df_obs.index)
            df_obs = df_obs.loc[df_obs.index.isin(period_overlap)]
            obs_dict[gsim_no]=df_obs

            ### SIMULATION
            # get simulated values for station and corresponding period
            seg_id = stations.loc[stations['gsim.no']==gsim_no, 'seg_id'].values[0]
            idx = ntopo['seg_id'].values.tolist().index(seg_id)

            case_mod_dict = {}
            metric_station_dict = {}

            for case in cases: 
                mod = ds_sim_dict[case]['discharge'].sel(seg=int(idx))

                mod_mean = mod.resample(time=freq).mean(dim='time')
                mod_sd   = mod.resample(time=freq).std(dim='time')
                mod_min  = mod.resample(time=freq).min(dim='time')
                mod_max  = mod.resample(time=freq).max(dim='time')
                mod_75   = mod.resample(time=freq).quantile(0.75,dim='time')
                mod_25    = mod.resample(time=freq).quantile(0.25,dim='time')
                
                # assign indices to dataframe with corresponding periods to obs
                df_mod = pd.DataFrame(columns=df_obs.columns,index=df_obs.index)
                idt = ds_sim_dict[case]['time'][sim_period.isin(period_overlap)]

                df_mod['MEAN'] = mod_mean.sel(time=idt)
                df_mod['SD']   = mod_sd.sel(time=idt)
                df_mod['CV']   = mod_sd.sel(time=idt) / mod_mean.sel(time=idt)
                df_mod['MIN']  = mod_min.sel(time=idt)
                df_mod['MAX']  = mod_max.sel(time=idt)
                df_mod['IQR']  = mod_75.sel(time=idt) - mod_25.sel(time=idt)

                case_mod_dict[case] = df_mod

                # create dataseries per index
                kge = pd.Series(np.empty(10),index = df_obs.keys() )
                pearson = pd.Series(np.empty(10),index = df_obs.keys() )
                alpha = pd.Series(np.empty(10),index = df_obs.keys() )
                beta = pd.Series(np.empty(10),index = df_obs.keys() )

                nse = pd.Series(np.empty(10),index = df_obs.keys() )
                pbias = pd.Series(np.empty(10),index = df_obs.keys() )
                pabias = pd.Series(np.empty(10),index = df_obs.keys() )
                rmse = pd.Series(np.empty(10),index = df_obs.keys() )
                mare = pd.Series(np.empty(10),index = df_obs.keys() )

                for index in df_obs.keys(): 
                    temp = df_obs[df_mod.notnull()]
                    obs_index = temp[temp[index].notna()][index].values

                    temp = df_mod[df_obs.notnull()]
                    mod_index = temp[temp[index].notna()][index].values
                    
                    if (obs_index.size==0 or mod_index.size==0): 
                        kge[index]   = np.nan
                        pearson[index]   = np.nan
                        alpha[index]   = np.nan
                        beta[index]   = np.nan
                        nse[index]   = np.nan
                        rmse[index]  = np.nan
                        pbias[index] = np.nan
                        pabias[index] = np.nan
                        mare[index]  = np.nan
                    else:    
                        kge[index], pearson[index], alpha[index],  beta[index] = he.evaluator(he.kge,obs_index,mod_index)
                        nse[index]   = he.evaluator(he.nse,obs_index,mod_index)[0]
                        rmse[index]  = he.evaluator(he.rmse,obs_index,mod_index)[0]
                        pbias[index] = he.evaluator(he.pbias,obs_index,mod_index)[0]
                        pabias[index] = np.sum(abs(mod_index - obs_index)/obs_index)) *100
                        
                        mare[index]  = he.evaluator(he.mare,obs_index,mod_index)[0]
           
                
                metric_station_dict[case] = pd.DataFrame(data={'NSE':nse,'RMSE':rmse,'PBIAS':pbias,'PABIAS': pabias,'KGE':kge, 'MARE':mare, 'pearson': pearson, 'alpha':alpha, 'beta': beta})

            mod_dict[gsim_no] = case_mod_dict
            metric_dict[gsim_no] = metric_station_dict
    metrics_percase_dict = pd.DataFrame(metric_dict).transpose().to_dict()


    # save dicts per stations
    import pickle
    f = open(datadir+"/gsim/processed/mod_dict_gsim.pkl","wb")
    pickle.dump(mod_dict,f)
    f.close()

    f = open(datadir+"/gsim/processed/metric_dict_gsim.pkl","wb")
    pickle.dump(metric_dict,f)
    f.close()

else: 
    mod_dict = pickle.load( open(datadir+"/gsim/processed/mod_dict_gsim.pkl", "rb" ) )
    metric_dict = pickle.load( open( datadir+"/gsim/processed/metric_dict_gsim.pkl", "rb" ) )


CPU times: user 22min 33s, sys: 720 ms, total: 22min 34s
Wall time: 22min 57s


## Filter only controlled stations
this is now done above. 

## calculate differences in metrics between cases

In [16]:
calc_differences = True

In [26]:
%%time
if calc_differences: 

    # calculate delta metrics
    metrics = ['NSE','KGE','RMSE', 'MARE','PBIAS','PABIAS', 'pearson', 'alpha', 'beta']
    metric_dnolak_dict = {}
    metric_dnatlak_dict = {}
    metric_dnatlak_nolak_dict = {}

    # delta

    for gsim_no in metric_dict.keys():
        metric_dnolak_dict[gsim_no]  = metric_dict[gsim_no][case_H06]-metric_dict[gsim_no][case_nolake]
        metric_dnatlak_dict[gsim_no] = metric_dict[gsim_no][case_H06]-metric_dict[gsim_no][case_natlake]
        metric_dnatlak_nolak_dict[gsim_no] = metric_dict[gsim_no][case_natlake]-metric_dict[gsim_no][case_nolake]


    # create dataframe for certain metric per case and per station
    df_metrics_perstation = pd.DataFrame({'gsim_no':list(metric_dict.keys())})
    metrics_dnolak_dict = {}
    metrics_dnatlak_dict = {}
    metrics_dnatlak_nolak_dict = {}
    for metric in metrics:
        col = pd.DataFrame()
        for gsim_no in metric_dict.keys(): 
            df_nolak = pd.DataFrame(metric_dnolak_dict[gsim_no][metric]).transpose()
            df_nolak['gsim_no'] = gsim_no
            col=col.append(df_nolak)
        metrics_dnolak_dict[metric] = df_metrics_perstation.merge(col,on='gsim_no',how='left')

        col_natlak = pd.DataFrame()
        for gsim_no in metric_dict.keys(): 
            df_natlak = pd.DataFrame(metric_dnatlak_dict[gsim_no][metric]).transpose()
            df_natlak['gsim_no'] = gsim_no
            col_natlak=col_natlak.append(df_natlak)
        metrics_dnatlak_dict[metric] = df_metrics_perstation.merge(col_natlak,on='gsim_no',how='left')


        col_natlak_nolak = pd.DataFrame()
        for gsim_no in metric_dict.keys(): 
            df_natlak_nolak = pd.DataFrame(metric_dnatlak_nolak_dict[gsim_no][metric]).transpose()
            df_natlak_nolak['gsim_no'] = gsim_no
            col_natlak_nolak=col_natlak_nolak.append(df_natlak_nolak)
        metrics_dnatlak_nolak_dict[metric] = df_metrics_perstation.merge(col_natlak_nolak,on='gsim_no',how='left')
        
    f = open(datadir+"/gsim/processed/metrics_dnolak_dict.pkl","wb")
    pickle.dump(metrics_dnolak_dict,f)
    f.close()

    f = open(datadir+"/gsim/processed/metrics_dnatlak_dict.pkl","wb")
    pickle.dump(metrics_dnatlak_dict,f)
    f.close()

else: 
    
    metrics_dnatlak_dict = pickle.load( open(datadir+"/gsim/processed/metric_dnatlak_dict.pkl", "rb" ) )
    metrics_dnolak_dict = pickle.load( open( datadir+"/gsim/processed/metric_dnolak_dict.pkl", "rb" ) )

CPU times: user 24.4 s, sys: 20.3 ms, total: 24.4 s
Wall time: 24.4 s


## Sort data in different ways

In [27]:
sort_indices = True

In [29]:
%%time

if sort_indices: 
# sort per index for all stations

    indices = ['MEAN', 'SD', 'CV', 'IQR', 'MIN', 'MAX', 'MIN7', 'MAX7']


    metrics_perindex_percase_dict = {}
    for case in cases: 
        metrics_perindex_dict = {}
        for index in indices:
            index_all_stations = []

            for station in metrics_percase_dict[case].keys(): 
                station_values = metrics_percase_dict[case][station].loc[index].values.squeeze()
                index_all_stations.append(station_values)
            metrics_allstations = pd.DataFrame(np.array(index_all_stations), index=metrics_percase_dict[case], columns= metrics_percase_dict[case][station].keys())
            metrics_perindex_dict[index] = metrics_allstations

        metrics_perindex_percase_dict[case] = metrics_perindex_dict
        
    f = open(datadir+"/gsim/processed/metrics_perindex_dict.pkl","wb")
    pickle.dump(metrics_perindex_dict,f)
    f.close()
    # sort per metric for all stations
    metrics = ['KGE', 'NSE','RMSE', 'MARE','PBIAS', 'PABIAS']

    indices_permetric_percase_dict = {}
    for case in cases: 
        indices_permetric_dict = {}
        for metric in metrics:
            metric_all_stations = []

            for station in metrics_percase_dict[case].keys(): 
                station_values = metrics_percase_dict[case][station][metric].values.squeeze()
                metric_all_stations.append(station_values)

            indices_allstations = pd.DataFrame(np.array(metric_all_stations), index=metrics_percase_dict[case], columns= metrics_percase_dict[case][station].index.values)
            indices_permetric_dict[metric] = indices_allstations

        indices_permetric_percase_dict[case] = indices_permetric_dict
        
    f = open(datadir+"/gsim/processed/indices_permetric_percase_dict.pkl","wb")
    pickle.dump(indices_permetric_percase_dict,f)
    f.close()
    
else: 
    metrics_perindex_dict = pickle.load( open(datadir+"/gsim/processed/metrics_perindex_dict.pkl", "rb" ) )
    indices_permetric_percase_dict = pickle.load( open(datadir+"/gsim/processed/indices_permetric_percase_dict.pkl", "rb" ) )


CPU times: user 1.35 s, sys: 0 ns, total: 1.35 s
Wall time: 1.35 s
